In [4]:
path = ""

In [3]:
import h5py

In [6]:
with h5py.File(path, "r") as f:
    print("Top-level keys:", list(f.keys()))
    # Example: read a dataset called 'X'
    if "X" in f:
        X = f["X"][()]   # NumPy array
        print(X.shape, X.dtype)

Top-level keys: ['#refs#', '#subsystem#', 'IsWatRef', 'NoiseCorrMatStruct', 'NoiseScalingMatrix_spectral', 'NoiseScalingMatrix_time', 'Par', 'csi', 'mask', 'weights']


In [1]:
import file

config_file_name = "paths_09012026.json"

configurator = file.Configurator(path_folder="../config/",
                                 file_name=config_file_name)
configurator.load()

In [38]:
import nibabel as nib
import numpy as np
import os

In [39]:
metabolite_map = nib.load(os.path.join(configurator.data["maps"]["metabolites_concentration"]["path"], "MetMap_Cr+PCr_con_map_TargetRes.nii"))
mask = nib.load(configurator.data["mask"]["metabolites"]["path"])

In [40]:
print(mask.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 112 128  80   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.     1.7188 1.7188 1.72   1.     0.     0.     0.    ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 1
glmin           : 0
descrip         : b'FSL4.0'
aux_file        : b'none'
qform_code      : unknown
sform_code      : scanner
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : -94.534
qoffset_y       : -108.28439
qoffset_z       : -67.08

In [41]:
c1 = metabolite_map.header["srow_x"][:3]
c2 = metabolite_map.header["srow_y"][:3]
c3 = metabolite_map.header["srow_z"][:3]

print(c1)
print(c2)
print(c3)

[ 1.2193333  -0.05813259  0.06051305]
[0.05625402 1.2203125  0.03866627]
[-0.06236025 -0.03584902  1.2181034 ]


In [45]:
np.cross(c1, c2) /np.linalg.norm(np.cross(c1, c2))

array([-0.05093832, -0.02928265,  0.99827236], dtype=float32)

In [50]:
np.array([c1[2], c2[2], c3[2]]) / np.linalg.norm([c1[2], c2[2], c3[2]])

array([0.04959198, 0.03168799, 0.9982667 ], dtype=float32)

### normal_vector_slice = $\frac{c_3}{||c_3||_2}$


### 3D Affine Transformation Matrices:

Any combination of translation, rotations, scalings/reﬂections and shears can be combined in a single 4 by 4 afﬁne transformation matrix.

$$
\begin{bmatrix}
x\\
y\\
z\\
1
\end{bmatrix}
=
\begin{bmatrix}
 & & & t_x\\
 & A & & t_y\\
 & & & t_z\\
0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
i\\
j\\
k\\
1
\end{bmatrix}
$$

$$
\begin{bmatrix}
x\\
y\\
z\\
1
\end{bmatrix}
=
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & t_x\\
a_{21} & a_{22} & a_{23} & t_y\\
a_{31} & a_{32} & a_{33} & t_z\\
0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
i\\
j\\
k\\
1
\end{bmatrix}
$$

* A = 3×3-Teil oben links (Rotation + Skalierung, evtl. Shear)
* $t=(t_x, t_y, t_z)$ = Translation (Position des Volumens im Raum)


$$
A\begin{bmatrix} i \\ j \\ k \end{bmatrix}
=
i\,A\begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}
+
j\,A\begin{bmatrix} 0 \\ 1 \\ 0 \end{bmatrix}
+
k\,A\begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}
$$


## Die Vektoren

- $\left[1,0,0\right]^T$
- $\left[0,1,0\right]^T$
- $\left[0,0,1\right]^T$

sind die **Basisrichtungen im Voxelraum** $(i,j,k)$.

## Wenn ich im Volumen Slice zu Slice geeh, dann
$(i,j,k) \rightarrow (i,j,k+1)$ mit $(i,j)$ im slice bewegen und $k$ zwischen slices,

wobei die differenz ist

$(i,j,k +1) - (i,j,k) = (0,0,1)$

und somit der vektor

$\Delta v = \begin{bmatrix}0\\0\\1\end{bmatrix}$

und somit kommt man auf die Weltkoordinaten via:

$r(i,j,k) = A \begin{bmatrix}i\\j\\k\end{bmatrix}+ t$ $~~~~~\longrightarrow~~~~~~$ $r(i,j,k+1) = A \begin{bmatrix}i\\j\\k+1\end{bmatrix}+ t$

und die Differenz ergibt sich aus:

$\Delta r = A \Biggl( \begin{bmatrix}i\\j\\k+1\end{bmatrix} - \begin{bmatrix}i\\j\\k\end{bmatrix} \Biggr) = A \begin{bmatrix}0\\0\\1\end{bmatrix}$

und

$$
A\begin{bmatrix}0\\0\\1\end{bmatrix}
=
\begin{bmatrix}
a_{11}\cdot 0 + a_{12}\cdot 0 + a_{13}\cdot 1\\
a_{21}\cdot 0 + a_{22}\cdot 0 + a_{23}\cdot 1\\
a_{31}\cdot 0 + a_{32}\cdot 0 + a_{33}\cdot 1
\end{bmatrix}
=
\begin{bmatrix}
a_{13}\\
a_{23}\\
a_{33}
\end{bmatrix}
$$

In [51]:
np.array([c1[2], c2[2], c3[2]]) / np.linalg.norm([c1[2], c2[2], c3[2]])

array([0.04959198, 0.03168799, 0.9982667 ], dtype=float32)

In [ ]:
# Affine Matrix

#In NIfTI beschreibt die affine Matrix (4×4), wie man von Voxel-Koordinaten (i,j,k) zu Welt-/Scanner-Koordinaten (x,y,z) kommt.

